In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score, roc_auc_score



# Charger les données
data = pd.read_csv("card_credit_fraud.csv", delimiter=';')

# Mapper les types
type_mapping = {'CASH_OUT': 1, 'PAYMENT': 2, 'CASH_IN': 3, 'TRANSFER': 4, 'DEBIT': 5}
data['type'] = data['type'].map(type_mapping)

# Supprimer les valeurs manquantes
data = data.dropna()

# Séparer les caractéristiques (X) de la cible (y)
X = data[['type', 'step', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']]
y = data["isFraud"]
seed = 42

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

# Créer le modèle Naive Bayes
model = GaussianNB()

# Sur-échantillonnage avec SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=seed, k_neighbors=7)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Entraîner le modèle sur les données sur-échantillonnées
model.fit(X_train_smote, y_train_smote)

# Prédire les classes pour l'ensemble de test
y_pred_smote = model.predict(X_test)

# Calculer la précision avec SMOTE
accuracy_smote = accuracy_score(y_test, y_pred_smote)
print("Précision du modèle équilibré avec SMOTE : {:.2f}%".format(accuracy_smote * 100))

# Sous-échantillonnage avec NearMiss en spécifiant la distribution de classe
nearmiss = NearMiss(sampling_strategy='auto')
X_under_sample, y_under_sample = nearmiss.fit_resample(X_train, y_train)

# Entraîner le modèle sur les données sous-échantillonnées
model.fit(X_under_sample, y_under_sample)

# Prédire les classes pour l'ensemble de test
y_pred_nearmiss = model.predict(X_test)

# Calculer la précision avec NearMiss
accuracy_nearmiss = accuracy_score(y_test, y_pred_nearmiss)
print("Précision du modèle équilibré avec NearMiss : {:.2f}%".format(accuracy_nearmiss * 100))

# Définir la grille des hyperparamètres à tester
param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
}

# Initialiser GridSearchCV pour NearMiss
grid_search = GridSearchCV(estimator=GaussianNB(), param_grid=param_grid, cv=5, scoring='f1')
grid_search.fit(X_under_sample, y_under_sample)

# Afficher les meilleurs hyperparamètres et le score correspondant pour NearMiss
print("Meilleurs hyperparamètres trouvés pour le NearMiss : ", grid_search.best_params_)

# Utiliser le meilleur modèle trouvé pour faire des prédictions sur les données de test avec NearMiss
best_model_nearmiss = grid_search.best_estimator_
y_pred_gridsearch_nearmiss = best_model_nearmiss.predict(X_test)

# Calculer la précision avec les meilleurs hyperparamètres pour NearMiss
accuracy_gridsearch_nearmiss = accuracy_score(y_test, y_pred_gridsearch_nearmiss)
print("Précision du modèle avec les meilleurs hyperparamètres pour NearMiss : {:.2f}%".format(accuracy_gridsearch_nearmiss * 100))

# Initialiser GridSearchCV pour SMOTE
grid_search_smote = GridSearchCV(estimator=GaussianNB(), param_grid=param_grid, cv=5, scoring='f1')
grid_search_smote.fit(X_train_smote, y_train_smote)

# Afficher les meilleurs hyperparamètres et le score correspondant pour SMOTE
print("Meilleurs hyperparamètres trouvés pour le SMOTE : ", grid_search_smote.best_params_)

# Utiliser le meilleur modèle trouvé pour faire des prédictions sur les données de test avec SMOTE
best_model_smote = grid_search_smote.best_estimator_
y_pred_gridsearch_smote = best_model_smote.predict(X_test)

# Calculer la précision avec les meilleurs hyperparamètres pour SMOTE
accuracy_gridsearch_smote = accuracy_score(y_test, y_pred_gridsearch_smote)
print("Précision du modèle avec les meilleurs hyperparamètres pour SMOTE : {:.2f}%".format(accuracy_gridsearch_smote * 100))

# Matrice de confusion pour le modèle avec sous-échantillonnage (NearMiss)
conf_matrix_nearmiss = confusion_matrix(y_test, y_pred_nearmiss)
print("Matrice de confusion pour le modèle avec sous-échantillonnage (NearMiss):")
print(conf_matrix_nearmiss)

# Matrice de confusion pour le modèle avec sur-échantillonnage (SMOTE)
conf_matrix_smote = confusion_matrix(y_test, y_pred_smote)
print("Matrice de confusion pour le modèle avec sur-échantillonnage (SMOTE):")
print(conf_matrix_smote)

# Calculer la F-mesure pour SMOTE
f1_smote = f1_score(y_test, y_pred_smote)
print("F-mesure pour le modèle avec SMOTE : {:.2f}".format(f1_smote))

# Calculer l'AUC-ROC pour SMOTE
auc_roc_smote = roc_auc_score(y_test, y_pred_smote)
print("AUC-ROC pour le modèle avec SMOTE : {:.2f}".format(auc_roc_smote))







Précision du modèle équilibré avec SMOTE : 95.38%
Précision du modèle équilibré avec NearMiss : 37.13%
Meilleurs hyperparamètres trouvés pour le NearMiss :  {'var_smoothing': 1e-09}
Précision du modèle avec les meilleurs hyperparamètres pour NearMiss : 37.13%
Meilleurs hyperparamètres trouvés pour le SMOTE :  {'var_smoothing': 1e-09}
Précision du modèle avec les meilleurs hyperparamètres pour SMOTE : 95.38%
Matrice de confusion pour le modèle avec sous-échantillonnage (NearMiss):
[[30633 53094]
 [   53   749]]
Matrice de confusion pour le modèle avec sur-échantillonnage (SMOTE):
[[79938  3789]
 [  113   689]]
F-mesure pour le modèle avec SMOTE : 0.26
AUC-ROC pour le modèle avec SMOTE : 0.91


In [12]:
import pickle

# Nom du fichier dans lequel vous souhaitez enregistrer le modèle
filename_smote = 'model_naive_bayes_smote.pkl'

# Enregistrer le modèle dans un fichier
with open(filename_smote, 'wb') as file:
    pickle.dump(best_model_smote, file)

 # Nom du fichier dans lequel vous souhaitez enregistrer le modèle
filename_nearmiss = 'model_naive_bayes_nearmiss.pkl'

# Enregistrer le modèle dans un fichier
with open(filename_nearmiss, 'wb') as file:
    pickle.dump(best_model_nearmiss, file)

